<a href="https://colab.research.google.com/github/mnocerino23/Wildfire-Forecaster/blob/main/Regression_LargeDataset_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Having found and selected features that provide good regression models using linear, ridge, lasso, and decision tree regressors, I will now try to tune the performance of these models

# In this Jupyter Notebook, I will use GridSearch Cross Validation in Scikit-learn to find the hyperparemeters that optimize the performance of theses models.



In [377]:
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

#Read in the smaller of the final datasets. The dataset contains around 1200 fires from 2016-2019
wildfire_set1 = pd.read_csv('/content/drive/MyDrive/Data_Science_Projects/Wildfires/wildfire_set1_w_allfeatures.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [378]:
wildfire_set1.head()

,Unnamed: 0,Year,Name,AcresBurned,Fire Size Rank,Cause,SOURCE_REPORTING_UNIT_NAME,DaysBurn,Discovery Month,Discovered DOY,...,DX90_2M,DP10_2M,Receives Snow,Snow Station,River Basin,Mar_SP,Mar_WC,Mar_Dens,Has_Elevation,Elevation
0,0,2005,FOUNTAIN,0.10,A,Miscellaneous,Plumas National Forest,1.0,Feb,33.0,...,0.0,19.0,1.0,eureka_lake,Feather,79.6,34.0,0.43,1,2965.8432
1,1,2004,PIGEON,0.25,A,Lightning,Eldorado National Forest,1.0,May,133.0,...,0.0,3.0,1.0,ward_creek_2,Lake Tahoe,108.6,38.1,0.35,1,6207.2736
2,2,2004,SLACK,0.10,A,Debris Burning,Eldorado National Forest,1.0,Jun,152.0,...,0.0,11.0,1.0,ward_creek_2,Lake Tahoe,108.6,38.1,0.35,1,3454.6824
3,3,2004,DEER,0.10,A,Lightning,Eldorado National Forest,5.0,Jun,180.0,...,0.0,3.0,1.0,echo_summit,American,87.2,28.4,0.33,1,7759.0920
4,4,2004,STEVENOT,0.10,A,Lightning,Eldorado National Forest,5.0,Jun,180.0,...,0.0,3.0,1.0,echo_summit,American,87.2,28.4,0.33,1,7598.3328


In [379]:
print(wildfire_set1.columns)

Index(['Unnamed: 0', 'Year', 'Name', 'AcresBurned', 'Fire Size Rank', 'Cause',
       'SOURCE_REPORTING_UNIT_NAME', 'DaysBurn', 'Discovery Month',
       'Discovered DOY', 'Contained Month', 'Contained DOY', 'Latitude',
       'Longitude', 'County', 'CountyIds', 'State', 'OWNER_DESCR',
       'NOAA Station', 'Link', 'AWND', 'CLDD', 'DP10', 'DX90', 'PRCP', 'TAVG',
       'TMAX', 'TMIN', 'PRCP_6M', 'PRCP_RS', 'DX90_2M', 'DP10_2M',
       'Receives Snow', 'Snow Station', 'River Basin', 'Mar_SP', 'Mar_WC',
       'Mar_Dens', 'Has_Elevation', 'Elevation'],
      dtype='object')


Drop all columns that will not be relevant for our regression task:Unnamed: 0, Year, Name, Cause, SOURCE_REPORTING UNIT NAME, DaysBurn, Discovered DOY, Contained Month, Contained DOY, Lat, Long, County Ids, State, OWNER_DESCR, NOAA Station, Link, Snow Station, River Basin

In [380]:
wildfire_set1.drop(columns = ['Unnamed: 0', 'Year', 'Name', 'Cause', 'Fire Size Rank',
                      'SOURCE_REPORTING_UNIT_NAME', 'DaysBurn','Contained Month', 'Discovered DOY',
                      'Contained DOY','Latitude','Longitude','County','CountyIds','State','OWNER_DESCR',
                      'NOAA Station', 'Link', 'Snow Station', 'River Basin','Has_Elevation'], inplace = True)

Taking a look at the dataset now that we have dropped the nonrelevant columns

In [381]:
wildfire_set1.head(5)

,AcresBurned,Discovery Month,AWND,CLDD,DP10,DX90,PRCP,TAVG,TMAX,TMIN,PRCP_6M,PRCP_RS,DX90_2M,DP10_2M,Receives Snow,Mar_SP,Mar_WC,Mar_Dens,Elevation
0,0.10,Feb,5.6,0.0,12.0,0.0,5.33,38.9,43.9,33.9,27.89,49.06,0.0,19.0,1.0,79.6,34.0,0.43,2965.8432
1,0.25,May,6.9,0.0,2.0,0.0,0.81,47.3,63.0,31.6,14.37,14.76,0.0,3.0,1.0,108.6,38.1,0.35,6207.2736
2,0.10,Jun,5.6,36.0,0.0,0.0,0.00,63.1,70.2,56.0,36.71,40.37,0.0,11.0,1.0,108.6,38.1,0.35,3454.6824
3,0.10,Jun,5.6,0.0,1.0,0.0,0.29,54.7,72.9,36.5,13.63,14.76,0.0,3.0,1.0,87.2,28.4,0.33,7759.0920
4,0.10,Jun,5.6,0.0,1.0,0.0,0.29,54.7,72.9,36.5,13.63,14.76,0.0,3.0,1.0,87.2,28.4,0.33,7598.3328


In [382]:
wildfire_set1.shape

(114558, 19)

# Investigate the presence of null values in the dataset:
Below, we have the count of totall null values in each column for  wildfire_set1.

In [383]:
print(wildfire_set1.isnull().sum())

AcresBurned           0
Discovery Month       0
AWND               6254
CLDD               3335
DP10               3042
DX90               3333
PRCP               3042
TAVG               3335
TMAX               3333
TMIN               3285
PRCP_6M            2470
PRCP_RS            2811
DX90_2M            2215
DP10_2M            1599
Receives Snow       102
Mar_SP              102
Mar_WC              102
Mar_Dens            102
Elevation             0
dtype: int64


In [384]:
wildfire_set1 = wildfire_set1.dropna()

In the cell above, we drop all null values in the dataset. We are able to drop all nulls in this case because our dataset is sizeable with relatively few null values in each so not much data is lost in this process. Looking at the shape of the dataframe after the drops, we see that the dataset remains a similar size post drop.

In [385]:
#we lose around 50 values when dropping NaN's
wildfire_set1.shape

(106024, 19)

# For our regressions, we will try to predict the number of Acres Burned in the fire

In [386]:
print(wildfire_set1.columns)

Index(['AcresBurned', 'Discovery Month', 'AWND', 'CLDD', 'DP10', 'DX90',
       'PRCP', 'TAVG', 'TMAX', 'TMIN', 'PRCP_6M', 'PRCP_RS', 'DX90_2M',
       'DP10_2M', 'Receives Snow', 'Mar_SP', 'Mar_WC', 'Mar_Dens',
       'Elevation'],
      dtype='object')


Double check our datatypes before we proceed with preprocessing and model building. As we can see, all features besides discovery and fire size rank are numerical (of type float or int) so all we have to do is one hot encode the discovery month of the fire.

In [387]:
wildfire_set1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106024 entries, 0 to 114557
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   AcresBurned      106024 non-null  float64
 1   Discovery Month  106024 non-null  object 
 2   AWND             106024 non-null  float64
 3   CLDD             106024 non-null  float64
 4   DP10             106024 non-null  float64
 5   DX90             106024 non-null  float64
 6   PRCP             106024 non-null  float64
 7   TAVG             106024 non-null  float64
 8   TMAX             106024 non-null  float64
 9   TMIN             106024 non-null  float64
 10  PRCP_6M          106024 non-null  float64
 11  PRCP_RS          106024 non-null  float64
 12  DX90_2M          106024 non-null  float64
 13  DP10_2M          106024 non-null  float64
 14  Receives Snow    106024 non-null  float64
 15  Mar_SP           106024 non-null  float64
 16  Mar_WC           106024 non-null  floa

# One-Hot Encoding Categorical Variables


*   Encode the discovery month (the only categorical variable) in both datasets



In [388]:
#Add the dummy columns to the dataset
dummy_month = pd.get_dummies(wildfire_set1['Discovery Month'])
wildfire_set1 = pd.merge(left = wildfire_set1, right = dummy_month, left_index = True, right_index = True)
wildfire_set1 = wildfire_set1.drop(columns = ['Discovery Month'])
wildfire_set1.head(10)

,AcresBurned,AWND,CLDD,DP10,DX90,PRCP,TAVG,TMAX,TMIN,PRCP_6M,...,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep
0,0.10,5.6,0.0,12.0,0.0,5.33,38.9,43.9,33.9,27.89,...,0,1,0,0,0,0,0,0,0,0
1,0.25,6.9,0.0,2.0,0.0,0.81,47.3,63.0,31.6,14.37,...,0,0,0,0,0,0,1,0,0,0
2,0.10,5.6,36.0,0.0,0.0,0.00,63.1,70.2,56.0,36.71,...,0,0,0,0,1,0,0,0,0,0
3,0.10,5.6,0.0,1.0,0.0,0.29,54.7,72.9,36.5,13.63,...,0,0,0,0,1,0,0,0,0,0
4,0.10,5.6,0.0,1.0,0.0,0.29,54.7,72.9,36.5,13.63,...,0,0,0,0,1,0,0,0,0,0
5,0.10,5.1,1.0,0.0,0.0,0.00,61.1,79.8,42.3,8.31,...,0,0,0,1,0,0,0,0,0,0
6,0.10,5.1,1.0,0.0,0.0,0.00,61.1,79.8,42.3,8.31,...,0,0,0,1,0,0,0,0,0,0
7,0.80,6.5,7.0,9.0,0.0,4.99,55.5,67.0,44.0,25.91,...,0,0,0,0,0,1,0,0,0,0
8,1.00,6.5,7.0,9.0,0.0,4.99,55.5,67.0,44.0,25.91,...,0,0,0,0,0,1,0,0,0,0
9,0.10,5.1,1.0,0.0,0.0,0.00,61.1,79.8,42.3,8.31,...,0,0,0,1,0,0,0,0,0,0


In [389]:
wildfire_set1.head()

,AcresBurned,AWND,CLDD,DP10,DX90,PRCP,TAVG,TMAX,TMIN,PRCP_6M,...,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep
0,0.10,5.6,0.0,12.0,0.0,5.33,38.9,43.9,33.9,27.89,...,0,1,0,0,0,0,0,0,0,0
1,0.25,6.9,0.0,2.0,0.0,0.81,47.3,63.0,31.6,14.37,...,0,0,0,0,0,0,1,0,0,0
2,0.10,5.6,36.0,0.0,0.0,0.00,63.1,70.2,56.0,36.71,...,0,0,0,0,1,0,0,0,0,0
3,0.10,5.6,0.0,1.0,0.0,0.29,54.7,72.9,36.5,13.63,...,0,0,0,0,1,0,0,0,0,0
4,0.10,5.6,0.0,1.0,0.0,0.29,54.7,72.9,36.5,13.63,...,0,0,0,0,1,0,0,0,0,0


In [390]:
#Write the clean, encoded, and randomly shuffled data to csv before continuing 
wildfire_set1.to_csv('wildfire1_regression_clean.csv')

Split the data into train-test sets

In [391]:
#We will train our models using the larger dataset

In [392]:
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler

In [393]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report, ConfusionMatrixDisplay

Utilize an 80-20 train test split on our large dataset

In [394]:
train_data, test_data = train_test_split(wildfire_set1, test_size = 0.2, random_state = 1)

Print the shape of training and testing datasets after the split to make sure we have done this correctly

In [395]:
print(train_data.shape)
print(test_data.shape)

(84819, 30)
(21205, 30)


# Feature Selection

In [396]:
from sklearn import linear_model

In [397]:
from sklearn.feature_selection import SelectKBest, f_regression

In [398]:
top_features_ordered = []

In [399]:
train_target = train_data['AcresBurned']
train_features = train_data.drop(columns = ['AcresBurned'])

In [400]:
test_target = test_data['AcresBurned']
test_features = test_data.drop(columns = ['AcresBurned'])

Select the 6 best features using sklearn's SelectKBest (leads to lowest MAE)

In [401]:
best_features = SelectKBest(f_regression, k=10)
k_best_data = best_features.fit_transform(train_features, train_target)
mask = best_features.get_support()
new_features = train_features.columns[mask]
for item in new_features:
  if item not in top_features_ordered:
    top_features_ordered.append(item)

In [402]:
count = 1
for item in top_features_ordered:
  print(count, '. ', item)
  count += 1

1 .  DP10
2 .  PRCP
3 .  TAVG
4 .  TMIN
5 .  PRCP_6M
6 .  DX90_2M
7 .  DP10_2M
8 .  Elevation
9 .  May
10 .  Oct


# Now we normalize the train and test sets independently. In the previous notebook, we found that the features PRCP_6M, TAVG, TMIN, DX90_2M, Jul, May, Oct help us produce the best model

In [403]:
#train_data_selected_X = train_features[top_features_ordered] k = 6, lasso MAE= 148
#train_data_selected_X = train_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Elevation','Aug','May','Oct']] linear mae = 131
#train_data_selected_X = train_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Elevation','May','Oct','Apr']] #linear mae = 132 
#train_data_selected_X = train_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Elevation','May','Oct']] linear mae = 130
#train_data_selected_X = train_features[['PRCP_6M', 'TMIN','DX90_2M','Elevation','Oct','May']] linear mae = 133
#train_data_selected_X = train_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Elevation','Jul','May','Oct']] linear mae = 130
#train_data_selected_X = train_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Apr','May','Oct']] #linear mae = 125

train_data_selected_X = train_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Jul','May','Oct']] #linear mae = 123

train_data_y = train_target

#test_data_selected_X = test_features[top_features_ordered]
#test_data_selected_X = test_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Elevation','Aug','May','Oct']]
#test_data_selected_X = test_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Elevation','May','Oct','Apr']]
#test_data_selected_X = test_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Elevation','May','Oct']]
#test_data_selected_X = test_features[['PRCP_6M','TMIN','DX90_2M','Elevation','Oct','May']]
#test_data_selected_X = test_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Elevation','Jul','May','Oct']]
#test_data_selected_X = test_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Apr','May','Oct']]

test_data_selected_X = test_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Jul','May','Oct']]

test_data_y = test_target


#crucial features - PRCP_6M, May, Oct, TMIN, TAVG, DX90_2M, elevation
#neutral - PRCP
#less important
#not important - Aug, Apr

In [404]:
print('Train Features:')
print('train_data_X dimensions: ', train_data_selected_X.shape)
print('train_data_y dimensions:', train_data_y.shape, '\n')

print('Test Features:')
print('test_data_X dimensions: ', test_data_selected_X.shape)
print('test_data_y dimensions:', test_data_y.shape, '\n')

Train Features:
train_data_X dimensions:  (84819, 7)
train_data_y dimensions: (84819,) 

Test Features:
test_data_X dimensions:  (21205, 7)
test_data_y dimensions: (21205,) 



Normalize data using min-max-scalar (every feature will normalized so that range of values is 0-1) after splitting into train and test

In [405]:
from sklearn.preprocessing import MinMaxScaler

In [406]:
#utilize minmax scaler normalization
scaler = MinMaxScaler()

#Normalize the train, validate, and test independently after splitting
train_data_X = scaler.fit_transform(train_data_selected_X)
#train_target = scaler(train_target)

test_data_X = scaler.fit_transform(test_data_selected_X)
#train_target = scaler(test_target)

# Fit the model using our normalized features to predict Acres Burned

In [407]:
multivariate_regression = linear_model.LinearRegression()
multivariate_regression.fit(train_data_X, train_data_y)

LinearRegression()

Model coefficients: (Beta's)

In [408]:
multivariate_regression.coef_

array([-148.539494  , -205.46045123,  456.50830964,  -13.6892896 ,
         10.33495093,  -51.04024601,  118.18250191])

Model intercept:

In [409]:
multivariate_regression.intercept_

-49.49346612388253

In [410]:
from sklearn import metrics
import numpy as np

In [411]:
y_pred_linear = multivariate_regression.predict(test_data_X)

In [412]:
print('Mean Squared Error: ', metrics.mean_squared_error(test_data_y, y_pred_linear), '\n')
print('Mean Absolute Error: ', metrics.mean_absolute_error(test_data_y, y_pred_linear), '\n')
print('Root Mean Squared Error: ', np.sqrt(metrics.mean_squared_error(test_data_y, y_pred_linear)))

Mean Squared Error:  1741618.5339609734 

Mean Absolute Error:  123.71031243185534 

Root Mean Squared Error:  1319.7039569391968


Now use GridSearchCV to find best values of alpha for Ridge and Lasso Regressions

# Ridge Regression:

In [413]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

In [414]:
#use gridsearchCV with alpha values 0-12 to find the alpha that produces the best model

ridge_search = GridSearchCV(Ridge(), {
    'alpha': [0.025,0.05,0.075,0.1,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3
              ,3.25,3.5,3.75,4,4.25,4.5,4.75,5,5.25,5.5,5.75,6,6.25,6.5,6.75,7,
              7.25,7.5,7.75,8,8.25,8.5,8.75,9,9.25,9.5,9.75,10,10.25,10.5, 10.75, 11, 
              11.25, 11.5, 11.75, 12]
}, cv = 5, scoring = 'neg_mean_absolute_error', return_train_score = False)
ridge_search.fit(train_data_X, train_data_y)
pd.DataFrame(ridge_search.cv_results_)[['param_alpha', 'params', 'mean_test_score','rank_test_score']].sort_values(by = ['rank_test_score'])[0:5]

,param_alpha,params,mean_test_score,rank_test_score
51,12,{'alpha': 12},-161.728186,1
50,11.75,{'alpha': 11.75},-161.729949,2
49,11.5,{'alpha': 11.5},-161.731725,3
48,11.25,{'alpha': 11.25},-161.733511,4
47,11,{'alpha': 11},-161.735309,5


In [415]:
ridge_search.best_params_

{'alpha': 12}

# Reproduce the tuned ridge regression and get the other evaluation metrics such as MSE and RMSE

In [416]:
ridge_regression = Ridge(alpha = 12)
ridge_regression.fit(train_data_X, train_data_y)

Ridge(alpha=12)

In [417]:
ridge_regression.coef_

array([-140.58102285, -150.73132995,  396.02387446,  -15.70587457,
         10.22890432,  -51.67707316,  119.18172407])

In [418]:
ridge_regression.intercept_

-47.03662518515797

In [419]:
y_pred_ridge = ridge_regression.predict(test_data_X)

# MAE for Ridge Regression is slightly lower

In [420]:
print('Mean Squared Error: ', metrics.mean_squared_error(test_data_y, y_pred_ridge), '\n')
print('Mean Absolute Error: ', metrics.mean_absolute_error(test_data_y, y_pred_ridge), '\n')
print('Root Mean Squared Error: ', np.sqrt(metrics.mean_squared_error(test_data_y, y_pred_ridge)))

Mean Squared Error:  1741648.8914379447 

Mean Absolute Error:  124.34611173831203 

Root Mean Squared Error:  1319.7154585129117


# Lasso Regression:

In [421]:
from sklearn.linear_model import Lasso

In [422]:
lasso_search = GridSearchCV(Lasso(), {
    'alpha': [0.025,0.05, 0.075,0.1, 0.125,0.15,0.2, 0.25,0.5,0.75,1.0,1.25,1.5,1.75,2.0, 2.25,2.5,2.75,
              3.0,3.25,3.5,3.75,4,4.25,4.5,4.75,5,5.25,5.5,5.75,6,6.25,6.5,
              6.75,7,7.25,7.5,7.75,8,8.25, 8.5, 8.75, 9, 9.25, 9.5, 9.75, 10, 10.25, 10.5, 10.75, 11, 
              11.25, 11.5, 11.75, 12, 12.25, 12.5, 12.75, 13]
}, cv = 5, scoring = 'neg_mean_absolute_error', return_train_score = False)
lasso_search.fit(train_data_X, train_data_y)
pd.DataFrame(lasso_search.cv_results_)[['param_alpha', 'params', 'mean_test_score','rank_test_score']].sort_values(by = ['rank_test_score'])[0:5]

,param_alpha,params,mean_test_score,rank_test_score
58,13,{'alpha': 13},-160.649931,1
56,12.5,{'alpha': 12.5},-160.649931,1
55,12.25,{'alpha': 12.25},-160.649931,1
54,12,{'alpha': 12},-160.649931,1
53,11.75,{'alpha': 11.75},-160.649931,1


In [425]:
lasso_search.best_params_

{'alpha': 10.25}

In [426]:
#building the optimal lasso model with alpha = 10.25
lasso_regression = Lasso(alpha = 10.25)
lasso_regression.fit(train_data_X, train_data_y)

Lasso(alpha=10.25)

In [427]:
lasso_regression.coef_

array([-0.,  0.,  0.,  0.,  0., -0.,  0.])

In [428]:
lasso_regression.intercept_

84.30695919546328

In [429]:
y_pred_lasso = lasso_regression.predict(test_data_X)

In [430]:
print('Mean Squared Error: ', metrics.mean_squared_error(test_data_y, y_pred_lasso), '\n')
print('Mean Absolute Error: ', metrics.mean_absolute_error(test_data_y, y_pred_lasso), '\n')
print('Root Mean Squared Error: ', np.sqrt(metrics.mean_squared_error(test_data_y, y_pred_lasso)))

Mean Squared Error:  1740940.1333282352 

Mean Absolute Error:  135.81701024175982 

Root Mean Squared Error:  1319.4469043232605


# Decision Tree Regressor

In [431]:
from sklearn.tree import DecisionTreeRegressor

In [432]:
decisiontree_search = GridSearchCV(DecisionTreeRegressor(), {
    'max_depth': [1,2,3,4,5],
    'max_features': [None,'sqrt','auto','log2']
}, cv = 5, scoring = 'neg_mean_absolute_error', return_train_score = False)
decisiontree_search.fit(train_data_X, train_data_y)
pd.DataFrame(decisiontree_search.cv_results_)[['param_max_depth','param_max_features', 'params', 'mean_test_score','rank_test_score']].sort_values(by = ['rank_test_score'])[0:5]

,param_max_depth,param_max_features,params,mean_test_score,rank_test_score
15,4,log2,"{'max_depth': 4, 'max_features': 'log2'}",-159.168214,1
19,5,log2,"{'max_depth': 5, 'max_features': 'log2'}",-159.446159,2
17,5,sqrt,"{'max_depth': 5, 'max_features': 'sqrt'}",-160.261242,3
4,2,None,"{'max_depth': 2, 'max_features': None}",-160.493808,4
6,2,auto,"{'max_depth': 2, 'max_features': 'auto'}",-160.493808,5


In [433]:
#best parameters for our decision tree regressor
decisiontree_search.best_params_

{'max_depth': 4, 'max_features': 'log2'}

In [434]:
decision_tree = DecisionTreeRegressor(max_depth= 4, max_features= 'log2')

In [435]:
decision_tree.fit(train_data_selected_X, train_data_y)

DecisionTreeRegressor(max_depth=4, max_features='log2')

In [436]:
y_pred_dt = decision_tree.predict(test_data_selected_X)

In [437]:
print('Mean Squared Error: ', metrics.mean_squared_error(test_data_y, y_pred_dt), '\n')
print('Mean Absolute Error: ', metrics.mean_absolute_error(test_data_y, y_pred_dt), '\n')
print('Root Mean Squared Error: ', np.sqrt(metrics.mean_squared_error(test_data_y, y_pred_dt)))

Mean Squared Error:  1743656.5962538852 

Mean Absolute Error:  135.1614930843049 

Root Mean Squared Error:  1320.4758976421665


Create a table titled wildfire_regressions_tuned to hold the results of our various regressions

In [438]:
wildfire_regressions_tuned = pd.DataFrame(columns = ['Model Type', 'Parameters', 'Features Used', 'Number of Features', 'MSE', 'MAE', 'RMSE'])

In [439]:
wildfire_regressions_tuned['Model Type'] = ['Multivariate Linear', 'Ridge', 'Lasso','Decision Tree']
wildfire_regressions_tuned['Parameters'] = ['', ridge_search.best_params_, lasso_search.best_params_, decisiontree_search.best_params_]
wildfire_regressions_tuned['Number of Features'] = [7,7,7,7]
wildfire_regressions_tuned['Features Used'] = [['PRCP_6M','TAVG','TMIN','DX90_2M','Jul','May','Oct'],
                                          ['PRCP_6M','TAVG','TMIN','DX90_2M','Jul','May','Oct'],
                                          ['PRCP_6M','TAVG','TMIN','DX90_2M','Jul','May','Oct'],
                                          ['PRCP_6M','TAVG','TMIN','DX90_2M','Jul','May','Oct']]

In [440]:
wildfire_regressions_tuned

,Model Type,Parameters,Features Used,Number of Features,MSE,MAE,RMSE
0,Multivariate Linear,,"[PRCP_6M, TAVG, TMIN, DX90_2M, Jul, May, Oct]",7,NaN,NaN,NaN
1,Ridge,{'alpha': 12},"[PRCP_6M, TAVG, TMIN, DX90_2M, Jul, May, Oct]",7,NaN,NaN,NaN
2,Lasso,{'alpha': 10.25},"[PRCP_6M, TAVG, TMIN, DX90_2M, Jul, May, Oct]",7,NaN,NaN,NaN
3,Decision Tree,"{'max_depth': 4, 'max_features': 'log2'}","[PRCP_6M, TAVG, TMIN, DX90_2M, Jul, May, Oct]",7,NaN,NaN,NaN


In [441]:
mse = []
mse.append(metrics.mean_squared_error(test_data_y, y_pred_linear))
mse.append(metrics.mean_squared_error(test_data_y, y_pred_ridge))
mse.append(metrics.mean_squared_error(test_data_y, y_pred_lasso))
mse.append(metrics.mean_squared_error(test_data_y, y_pred_dt))

mae = []
mae.append(metrics.mean_absolute_error(test_data_y, y_pred_linear))
mae.append(metrics.mean_absolute_error(test_data_y, y_pred_ridge))
mae.append(metrics.mean_absolute_error(test_data_y, y_pred_lasso))
mae.append(metrics.mean_absolute_error(test_data_y, y_pred_dt))

rmse = []
rmse.append(np.sqrt(metrics.mean_squared_error(test_data_y, y_pred_linear)))
rmse.append(np.sqrt(metrics.mean_squared_error(test_data_y, y_pred_ridge)))
rmse.append(np.sqrt(metrics.mean_squared_error(test_data_y, y_pred_lasso)))
rmse.append(np.sqrt(metrics.mean_squared_error(test_data_y, y_pred_dt)))

In [442]:
wildfire_regressions_tuned['MSE'] = mse
wildfire_regressions_tuned['MAE'] = mae
wildfire_regressions_tuned['RMSE'] = rmse

# Below we have a table with the final regression model results that help us predict acres burned by a wildfire given input features PRCP_6M, TAVG, TMIN, DX90_2M, Jul, May, Oct

In [443]:
wildfire_regressions_tuned.sort_values(by = ['MAE']).reset_index(drop = True)

,Model Type,Parameters,Features Used,Number of Features,MSE,MAE,RMSE
0,Multivariate Linear,,"[PRCP_6M, TAVG, TMIN, DX90_2M, Jul, May, Oct]",7,1.741619e+06,123.710312,1319.703957
1,Ridge,{'alpha': 12},"[PRCP_6M, TAVG, TMIN, DX90_2M, Jul, May, Oct]",7,1.741649e+06,124.346112,1319.715459
2,Decision Tree,"{'max_depth': 4, 'max_features': 'log2'}","[PRCP_6M, TAVG, TMIN, DX90_2M, Jul, May, Oct]",7,1.743657e+06,135.161493,1320.475898
3,Lasso,{'alpha': 10.25},"[PRCP_6M, TAVG, TMIN, DX90_2M, Jul, May, Oct]",7,1.740940e+06,135.817010,1319.446904
